### **FACT ORDERS**

**Data Reading**

In [0]:
df = spark.sql("select * from databricks_cata.silver.orders")

In [0]:
df.limit(10).display()

In [0]:
%sql
SELECT * FROM databricks_cata.gold.dimproducts

In [0]:
df_dimcus = spark.sql("select DimCustomerKey, customer_id as dim_customer_id from databricks_cata.gold.dimcustomers")

df_dimpro = spark.sql("select product_id as DimProductKey, product_id as dim_product_id from databricks_cata.gold.dimproducts")

**Fact Dataframe**

In [0]:
df_fact = df.join(df_dimcus, df['customer_id']==df_dimcus['dim_customer_id'],how='left') \
            .join(df_dimpro, df['product_id']==df_dimpro['dim_product_id'],how='left')

df_fact_new = df_fact.drop('dim_customer_id','dim_product_id','customer_id','product_id')

In [0]:
df_fact_new.display()

**Upsert on Fact table**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databricks_cata.gold.FactOrders"):
    
    dlt_obj = DeltaTable.forName(spark, "databricks_cata.gold.FactOrders")
    
    dlt_obj.alias("trg").merge(df_fact_new.alias("src"),"trg.order_id = src.order_id and trg.DimCustomerKey=src.DimCustomerKey AND trg.DimProductKey=src.DimProductKey") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
    
else:
    df_fact_new.write.format("delta") \
            .option("mode","append") \
            .option("path","abfss://gold@databricksend2end.dfs.core.windows.net/FactOrders") \
            .saveAsTable("databricks_cata.gold.FactOrders")

In [0]:
%sql
SELECT * FROM databricks_cata.gold.factorders